In [30]:
import nltk
import os
import string
from HMM import supervised_HMM, unsupervised_HMM, HiddenMarkovModel
import re
import numpy as np

In [39]:
def parse_observations(lines):

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []

        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1

            # Add the encoded word.
            obs_elem.append(obs_map[word])

        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [31]:
def unsupervised_learning(lines, n_states, n_iters):
    '''
    n_iters: Number of iterations we should go through.
    n_states: Number of hidden states our HMM should have.
    '''
    # Train the HMM.
    obs, obs_map =  parse_observations(lines)
    flat_lines = [[item] for sublist in lines for item in sublist]
    leHMM = unsupervised_HMM(obs, n_states, n_iters)
    return leHMM, obs,obs_map 
    

In [32]:
def load_poems(files):
    
    lines = [] # 2d dictionary, each array is a split + cleaned line
    words = {} # dictionary of a word, and its frequency
    
    for f in files:
        file = open(f, 'r')
        for line in file:
            line = line.strip()
            if  len(line) < 10:
                # Too short to be a valid line
                continue
            line = "".join(l for l in line if l not in string.punctuation)
            line = line.lower()
            line = line.split()

            lines.append(line)

            for word in line:
                try:
                    # add to frequency if the word is already in the dic
                    words[word] += 1
                except KeyError:
                    # if not, add the word to the dic
                    words[word] = 1
    return lines, words

In [42]:
def save_HMM(hmmmmmm, filename):
    
    with open(filename+".txt", "w+") as filept:
        filept.write(str(hmmmmmm.L)+"\n")
        filept.write(str(hmmmmmm.D)+"\n")
        for i in hmmmmmm.A:
            line = ""
            for j in i:
                line += str(j) + ","
            filept.write(line[:len(line)-1]+"\n")
        for i in hmmmmmm.O:
            line = ""
            for j in i:
                line += str(j) + ","
            filept.write(line[:len(line)-1]+"\n")
        

def read_HMM(filename):
    with open(filename+".txt", "r") as filept:
        L = int(filept.readline())
        D = int(filept.readline())
        O = []
        A = []
        for i in range(L):
            line = [float(x) for x in filept.readline().split(",")]
            A.append(line)
        for j in range(L):
            line = [float(x) for x in filept.readline().split(",")]
            O.append(line)
    return HiddenMarkovModel(A, O)

In [37]:
file = "data/shakespeare.txt"
file2 = "data/spenser.txt"
lines, words = load_poems([file, file2])

In [40]:
HMM = unsupervised_learning(lines, 4, 10)

merp 0
merp 1
merp 2
merp 3
merp 4
merp 5
merp 6
merp 7
merp 8
merp 9


In [44]:
save_HMM(HMM, "10-iter-4-hidden-hmm-additional")

AttributeError: 'tuple' object has no attribute 'L'

In [21]:
def load_stress(all_words):
    stress = nltk.corpus.cmudict.dict()
    stress_dict = {}
    unclear = []


    for word in all_words:
        if word not in stress.keys():
            unclear.append(word)
        else:
            stress_dict[word] = stress[word]

    for word in stress_dict.keys():
        phoneme = stress_dict[word][0]
        syls = []

        for phon in phoneme:
            if '0' in phon:
                syls.append(0)
            elif '1' in phon:
                syls.append(1)

        stress_dict[word] = syls

    return stress_dict, unclear

In [22]:
all_words = list(words.keys())
# for the stress, just use the first thing from the array
stress_dict, unclear = load_stress(all_words) 